In [1]:
import os
from os.path import join as pjoin
from tqdm import tqdm
import numpy as np

In [2]:
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import mpl_toolkits.mplot3d.axes3d as p3
def plot_3d_motion(save_path, kinematic_tree, joints, title, figsize=(10, 10), fps=120, radius=4):
#     matplotlib.use('Agg')

    title_sp = title.split(' ')
    if len(title_sp) > 10:
        title = '\n'.join([' '.join(title_sp[:10]), ' '.join(title_sp[10:])])
    def init():
        ax.set_xlim3d([-radius / 2, radius / 2])
        ax.set_ylim3d([0, radius])
        ax.set_zlim3d([0, radius])
        # print(title)
        fig.suptitle(title, fontsize=20)
        ax.grid(b=False)

    def plot_xzPlane(minx, maxx, miny, minz, maxz):
        ## Plot a plane XZ
        verts = [
            [minx, miny, minz],
            [minx, miny, maxz],
            [maxx, miny, maxz],
            [maxx, miny, minz]
        ]
        xz_plane = Poly3DCollection([verts])
        xz_plane.set_facecolor((0.5, 0.5, 0.5, 0.5))
        ax.add_collection3d(xz_plane)

    #         return ax

    # (seq_len, joints_num, 3)
    data = joints.copy().reshape(len(joints), -1, 3)
    fig = plt.figure(figsize=figsize)
    # ax = p3.Axes3D(fig)
    ax = fig.add_subplot(111, projection='3d')
    init()
    MINS = data.min(axis=0).min(axis=0)
    MAXS = data.max(axis=0).max(axis=0)
    colors = ['red', 'blue', 'black', 'red', 'blue',  
              'darkblue', 'darkblue', 'darkblue', 'darkblue', 'darkblue',
             'darkred', 'darkred','darkred','darkred','darkred']
    frame_number = data.shape[0]
    #     print(data.shape)

    height_offset = MINS[1]
    data[:, :, 1] -= height_offset
    trajec = data[:, 0, [0, 2]]
    
    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

    #     print(trajec.shape)

    def update(index):
        #         print(index)
        # ax.lines = []
        # ax.collections = []
        ax.clear()
        # ax.set_xlim3d([-radius / 2, radius / 2])
        # ax.set_ylim3d([0, radius])
        # ax.set_zlim3d([0, radius])
        ax.view_init(elev=120, azim=-90)
        ax.dist = 7.5
        #         ax =
        plot_xzPlane(MINS[0]-trajec[index, 0], MAXS[0]-trajec[index, 0], 0, MINS[2]-trajec[index, 1], MAXS[2]-trajec[index, 1])
#         ax.scatter(data[index, :22, 0], data[index, :22, 1], data[index, :22, 2], color='black', s=3)
        
        if index > 1:
            ax.plot3D(trajec[:index, 0]-trajec[index, 0], np.zeros_like(trajec[:index, 0]), trajec[:index, 1]-trajec[index, 1], linewidth=1.0,
                      color='blue')
        #             ax = plot_xzPlane(ax, MINS[0], MAXS[0], 0, MINS[2], MAXS[2])
        
        
        for i, (chain, color) in enumerate(zip(kinematic_tree, colors)):
#             print(color)
            if i < 5:
                linewidth = 4.0
            else:
                linewidth = 2.0
            ax.plot3D(data[index, chain, 0], data[index, chain, 1], data[index, chain, 2], linewidth=linewidth, color=color)
        #         print(trajec[:index, 0].shape)

        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])


    ani = FuncAnimation(fig, update, frames=frame_number, interval=1000/fps, repeat=False)

    ani.save(save_path, fps=fps)
    plt.close()


In [3]:
src_dir = './HumanML3D/new_joints/'
tgt_ani_dir = "./HumanML3D/animations/"

In [10]:
# Right leg, Left leg, Spine, Right arm, Left arm
kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]
kinematic_chain_sep = [[k] for k in kinematic_chain]
print(kinematic_chain_sep)
os.makedirs(tgt_ani_dir, exist_ok=True)

[[[0, 2, 5, 8, 11]], [[0, 1, 4, 7, 10]], [[0, 3, 6, 9, 12, 15]], [[9, 14, 17, 19, 21]], [[9, 13, 16, 18, 20]]]


In [13]:
npy_files = os.listdir(src_dir)
npy_files = sorted(npy_files)
npy_files = npy_files[:10]

This will take a few hours for the whole dataset. Here we show ten animations for an example


To accelerate the process, you could copy and run this script.

In [14]:
for npy_file in tqdm(npy_files):
    data = np.load(pjoin(src_dir, npy_file))
    
    txt_src_dir = src_dir.replace('new_joints', 'texts')
    txt_file = npy_file.replace('.npy', '.txt')
    with open(pjoin(txt_src_dir, txt_file), 'r') as f:
        text_data = f.read()
    result_text = ""
    for line in text_data.split('\n'):
        result_text += line.split('#')[0] + '\n'

    save_path = pjoin(tgt_ani_dir, npy_file[:-3] + 'mp4')
    # if os.path.exists(save_path):
    #     print(f"Skipping {save_path} because it already exists")
    #     continue
#   You may set the title on your own.
    plot_3d_motion(save_path, kinematic_chain, data, title=result_text, fps=20, radius=4)
    if npy_file == npy_files[0]:
        for idx, k in enumerate(kinematic_chain_sep):
            save_path = pjoin(tgt_ani_dir, npy_file[:-3] + f'_kc_{idx}.mp4')
            plot_3d_motion(save_path, k, data, title=result_text, fps=20, radius=4)


100%|██████████| 10/10 [01:55<00:00, 11.51s/it]


In [41]:
data

array([[[ 0.00000000e+00,  8.36999297e-01,  0.00000000e+00],
        [ 8.05792287e-02,  7.73063481e-01,  6.59070909e-03],
        [-3.26472297e-02,  7.32079804e-01,  6.40355051e-04],
        ...,
        [-2.11191803e-01,  9.37251627e-01,  2.57635474e-01],
        [ 2.24863440e-01,  7.79130399e-01,  2.99875438e-01],
        [-5.39650917e-02,  7.53495812e-01,  3.77413511e-01]],

       [[-2.38920166e-03,  8.36332381e-01,  1.14154967e-03],
        [ 7.82898366e-02,  7.72806168e-01,  1.00609064e-02],
        [-3.45600881e-02,  7.31264353e-01,  1.43107411e-03],
        ...,
        [-2.24821076e-01,  9.37355042e-01,  2.46336639e-01],
        [ 2.06301197e-01,  7.67926097e-01,  3.00089061e-01],
        [-7.41223395e-02,  7.47140527e-01,  3.64410579e-01]],

       [[ 3.17509053e-04,  8.37339103e-01,  5.59732318e-03],
        [ 8.06139931e-02,  7.74207592e-01,  1.94221213e-02],
        [-3.13174762e-02,  7.32118845e-01,  4.08634869e-03],
        ...,
        [-2.52134740e-01,  9.41310346e-01,